# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

IMPORT TEXT from Terminal:

use uk_food

mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray

In [68]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [69]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [70]:
# confirm that our new database was created
showall = mongo.list_database_names()

showall

['admin', 'autosaurus', 'classDB', 'config', 'gardenDB', 'local', 'uk_food']

In [71]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [72]:
# review the collections in our new database

db.list_collection_names()


['establishments']

In [73]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('64ef79f9d715f5bd7046300c'),
 'FHRSID': 254250,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'PostCode': 'CT15 6DZ',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2021-08-17T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.383298', 'latitude': '51.148133'},
 'RightToReply': '',
 'Distance': 4587.362402580997,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCoun

In [74]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [76]:
# Create a dictionary for the new restaurant data
rest_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [77]:
# Insert the new restaurant into the collection
result = establishments.insert_one(rest_data)

In [78]:
# Check that the new restaurant was inserted
db.establishments.find_one(sort=[('_id', -1)])

{'_id': ObjectId('64ef7adbe36f380708224b35'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [79]:
# Find all unique BusinessType values
bus_types = establishments.distinct("BusinessType")

# Print header
print("BusinessType\tBusinessTypeID")

# Find each unique BusinessType and its corresponding BusinessTypeID
types_data = []
for business_type in bus_types:
    document = establishments.find_one({"BusinessType": business_type})
    if document:
        business_type_id = document.get("BusinessTypeID")
        types_data.append({"BusinessType": business_type, "BusinessTypeID": business_type_id})
        print(f"{business_type}\t{business_type_id}")

BusinessType	BusinessTypeID
Caring Premises	5
Distributors/Transporters	7
Farmers/growers	7838
Hotel/bed & breakfast/guest house	7842
Importers/Exporters	14
Manufacturers/packers	7839
Mobile caterer	7846
Other catering premises	7841
Pub/bar/nightclub	7843
Restaurant/Cafe/Canteen	1
Retailers - other	4613
Retailers - supermarkets/hypermarkets	7840
School/college/university	7845
Takeaway/sandwich shop	7844


3. Update the new restaurant with the `BusinessTypeID` you found.

In [80]:
# Define the query to match the document
query = {"BusinessName": "Penang Flavours"}

# Update the new restaurant with the correct BusinessTypeID
newid = {"$set": {"BusinessTypeID": 1}}  

# Update the document
update_result = establishments.update_one(query, newid)

In [81]:
# Confirm that the new restaurant was updated
establishments.find_one({"BusinessName": "Penang Flavours"})

{'_id': ObjectId('64ef7adbe36f380708224b35'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [82]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName': 'Dover'})

994

In [83]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})


In [84]:
# Check if any remaining documents include Dover
establishments.count_documents({'LocalAuthorityName': 'Dover'})

0

In [85]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('64ef79f9d715f5bd704632f3'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': '4',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [88]:
# Change the data type from String to Decimal for longitude and latitude
update_result = establishments.update_many({}, [{
    '$set': {
        'geocode.longitude': {'$toDouble': '$geocode.longitude'},
        'geocode.latitude': {'$toDouble': '$geocode.latitude'}
    }
}])

print(f"Updated {update_result.modified_count} documents.")


Updated 38785 documents.


Use `update_many` to convert `RatingValue` to integer numbers.

In [90]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [94]:
# Change the data type from String to Integer for RatingValue
# Change the data type from String to Decimal for longitude and latitude
update_result = establishments.update_many({}, [{
    '$set': {
        'RatingValue': {'$toInt': '$RatingValue'},
        'RatingValue': {'$toInt': '$RatingValue'}
    }
}])

print(f"Updated {update_result.modified_count} documents.")


Updated 34694 documents.


In [109]:
# Check that the coordinates and rating value are now numbers
cursor = establishments.find({}, {'geocode.longitude': 1, 'geocode.latitude': 1, 'RatingValue': 1, '_id': 0}).limit(10)

for r in cursor:
    print(r)

{'RatingValue': 4, 'geocode': {'longitude': 1.196408, 'latitude': 51.086058}}
{'RatingValue': 5, 'geocode': {'longitude': 1.195625, 'latitude': 51.083812}}
{'RatingValue': 5, 'geocode': {'longitude': 1.188537, 'latitude': 51.08084}}
{'RatingValue': 5, 'geocode': {'longitude': 1.188537, 'latitude': 51.08084}}
{'RatingValue': 4, 'geocode': {'longitude': 1.188537, 'latitude': 51.08084}}
{'RatingValue': 5, 'geocode': {'longitude': 1.194762, 'latitude': 51.085797}}
{'RatingValue': 5, 'geocode': {'longitude': 1.188537, 'latitude': 51.08084}}
{'RatingValue': 5, 'geocode': {'longitude': 1.18590330311705, 'latitude': 51.0783519967076}}
{'RatingValue': 5, 'geocode': {'longitude': 1.18590330311705, 'latitude': 51.0783519967076}}
{'RatingValue': 5, 'geocode': {'longitude': 1.18590330311705, 'latitude': 51.0783519967076}}
